# Wifi Setup

This notebook walks through how to setup for connection currently (and in the future, after reboots) to a wifi network.

This notebook assumes you have a wifi dongle plugged into your board and a basic wifi config file in in `/etc/netplan/02-wifi-setup.yaml`

#### Helper Functions
Below are helper and library imports that we will use to setup the wifi.

In [1]:
import yaml
import subprocess as sp
import json, os

def get_interface():
    interface = sp.getoutput("ifconfig | awk '$1 ~ /^wl/ {print $1}' | rev | cut -c 2- | rev")
    return interface

def get_ip(interface: str):
    ip_addr = sp.getoutput(f"ifconfig " + interface + " | grep 'inet ' | cut -f1 | awk '{ print $2 }'")
    return ip_addr

# Don't change these without changing bash commands below
INPUT_NETPLAN='/etc/netplan/02-wifi-setup.yaml'
OUTPUT_NETPLAN='/tmp/02-wifi-setup.yaml'

class DoubleQuoted(str):
    pass

def double_quoted_presenter(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style="\"")

yaml.add_representer(DoubleQuoted, double_quoted_presenter)

def add_new_access_point(ssid, pwd):
    with open(INPUT_NETPLAN,'r') as f:
        output = yaml.safe_load(f)
    access_points = output['network']['wifis']['wlan0']['access-points']
    if ssid in access_points.keys():
        print("Wifi already added, updating password")
        output['network']['wifis']['wlan0']['access-points'][ssid]['password'] = pwd
    else:
        print("Adding new wifi configuration")
        output['network']['wifis']['wlan0']['access-points'][ssid] = {'password': pwd}
    print("Currently configured access points are:")
    access_points = output['network']['wifis']['wlan0']['access-points']
    quoted_access_points = {}
    for (k, v) in access_points.items():
        quoted_access_points[DoubleQuoted(k)] = {'password': DoubleQuoted(v['password'])}
        print(f"\tSSID=\"{k}\", PWD=\"{v['password']}\"")
    output['network']['wifis']['wlan0']['access-points'] = quoted_access_points

    with open(OUTPUT_NETPLAN, 'w') as f:
        yaml.dump(output, f, sort_keys=False, default_flow_style=False)

#### Please enter some information about your wifi

In [ ]:
ssid = input("Type in the SSID:")
pwd = input("Type in the password:")

#### Now we will connect the board to your wifi

Note that you may see a lot of output!

In [ ]:
add_new_access_point(ssid, pwd)

In [ ]:
%%bash

sudo cp /tmp/02-wifi-setup.yaml /etc/netplan/02-wifi-setup.yaml
sudo chmod 600 /etc/netplan/02-wifi-setup.yaml
sudo netplan generate
sudo netplan apply

#### Check Connectivity and Get IP Address and Interface

Note that if you got an error here, you probably did not connect correctly for some reason and should troubleshoot! You may also need to a wait a minute before an IP address is assigned.

In [ ]:
my_interface = get_interface()
my_ip = get_ip(my_interface)
print(f"My interface is {my_interface} and my IP is {my_ip}")

# OLED IP Address Display Setup

Now we will install a service that will print the IP address assigned to the wifi interface every time the board boots up, assuming there is an OLED display attached.

Please note you only need to do this once per board setup, not once per wifi setup!

In [ ]:
cmd = "cd /home/ubuntu/show-kria-ip && sudo ./setup.sh"
print("Running: " + cmd)
output = sp.getoutput(cmd)
lines = output.split("\n")
for l in lines:
    print(l)